In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from bokeh.palettes import viridis, brewer, magma, d3, cividis
from bokeh.models import HoverTool, LinearColorMapper
from bokeh.models.tickers import FixedTicker
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import output_notebook, reset_output, output_file, save

import sys
sys.path.append('../..')

from src.visualization.visualize import *
from src.data.make_dataset import *

output_notebook()

## Read in and merge data

In [ ]:
df = read_and_merge_data()

## Generate fake data for viz purposes

In [ ]:
df = make_fake_data(df)

## Visualization

In [ ]:
source = make_dataset(df)
p = make_plot(source)

In [ ]:
show(p)

In [ ]:
output_file("../../data/output/matrix.html")
save(p)
reset_output()